In [40]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
import xarray as xr
import pathlib
import sys
import datetime as dt
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pickle

ROOT_PATH = pathlib.Path().resolve().parent
ROOT_PATH

sys.path.append(str(ROOT_PATH))

from modules.date_range import date_range


start = dt.date(2022,1,1)
end = dt.date(2023,1,1)
datasets = [xr.open_dataset(date.strftime(f"{ROOT_PATH}/data/%Y/%m/%Y_%m%d.nc"))[['psea', 'sp', 'u', 'v', 'temp', 'rh', 'r1h', 'dswrf','ncld']].dropna(dim="time") for date in date_range(start,end)]
print("dataset Success")
del start,end


combined_dataset = xr.concat(datasets,dim="time")
print("combine Success")

del datasets


# 特徴量とターゲット変数に分割する
X = combined_dataset[['psea', 'sp', 'u', 'v', 'temp', 'rh', 'r1h', 'dswrf']]
y = combined_dataset['ncld']

del combined_dataset


X_df = X.to_dataframe()
print("X_df Success")
del X


y_df = y.to_dataframe()
print("y_df Success")
del y


# データセットをトレーニングセットとテストセットに分割する
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.3)

del X_df,y_df
del X_train,y_train

dataset Success
combine Success
X_df Success
y_df Success


In [5]:
with open(ROOT_PATH/"model_using_msm.pkl", 'rb') as f:
    model = pickle.load(f)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

y_pred = model.predict(X_test)

In [35]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

rmse = np.sqrt(mean_squared_error(y_test["ncld"].values,y_pred))
r2 = r2_score(y_test["ncld"].values,y_pred)

In [37]:
print(f"RMSE: {rmse}")
print(f"R2: {r2}")

RMSE: 23.543900686948984
R2: 0.45154444286971285
